In [ ]:
import math
import pandas as pd
import numpy as np
import requests
import json
import matplotlib.pyplot as plt


In [ ]:
!pip install neo4j
from neo4j import GraphDatabase

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.7/187.7 kB 4.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j: filename=neo4j-5.10.0-py3-none-any.whl size=259296 sha256=7056fdde27428c136b704dc91a25afea1c7691683cf6b230364cc37624373ef0
  Stored in directory: /root/.cache/pip/wheels/14/9e/82/738a3a6462074384260415f8f7c3834e9309372ff627f04a2d
Successfully built neo4j


In [ ]:

def haversine_distance(lat1, lon1, depth1, lat2, lon2, depth2):

    # Pasamos lat y long a radianes
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Diámetro de la tierra
    radius = 6371.0

    # Calculamos diferencias
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad
    ddepth = depth2 - depth1

    # Cruzamos los dedos
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    distance = radius * c

    # pitágoras
    distance = math.sqrt(distance ** 2 + ddepth ** 2)

    return distance


In [ ]:
df_ciudades = pd.read_csv("ciudades_california.csv")
df_ciudades.head()

,City,Latitude,Longitude
0,Los Angeles,34.0522,-118.2437
1,San Diego,32.7157,-117.1611
2,San Jose,37.3382,-121.8863
3,San Francisco,37.7749,-122.4194
4,Fresno,36.7372,-119.7871


In [ ]:
file_name = 'California.txt'


def get_earthquake_data(start_date, end_date):
    url = "https://earthquake.usgs.gov/fdsnws/event/1/query"

    # Parameters for the API request
    params = {
        "format": "geojson",
        "starttime": start_date,
        "endtime": end_date,
#        "maxlatitude" : -30,
#        "minlatitude" : -35,
#        "minlongitude" : -170.44,
#        "maxlongitude" : -53.24
#        "maxlatitude" : -15, #chile
#        "minlatitude" : -75, #chile
#        "minlongitude" : -75, #chile
#        "maxlongitude" : -66  #chile
        "maxlatitude" : 44, #California
        "minlatitude" : 30, #California
        "minlongitude" : -125, #California
        "maxlongitude" : -114  #California
        #"includeallorigins": "true"
    }

    # Sending the API request and getting the response
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print("Error occurred while fetching earthquake data.")
        print(response)
        return None

# Specify the start and end date for the data retrieval
#start_date = "2019-07-01"
#end_date = "2019-07-02"
start_date = "2023-05-20"
end_date = "2023-05-30"

# Fetch earthquake data for the specified date range
earthquake_data = get_earthquake_data(start_date, end_date)



In [ ]:
# Process the retrieved data
cols = ['donde','lat','long','prof','t']
df_terrem = pd.DataFrame(columns = cols)

if earthquake_data:
    # Extract relevant information from the retrieved data
    features = earthquake_data["features"]

    for feature in features:
        properties = feature["properties"]
        geometry = feature["geometry"]
        place = properties["place"]
        magnitude = properties["mag"]
        time = properties["time"]
        coords = geometry["coordinates"]

       # You can extract and print other relevant information as well
        new_row = [[place,coords[1],coords[0],coords[2],time]]
        new_df = pd.DataFrame(new_row, columns=cols)
        #new_row = {}
        #df_terrem = df_terrem.append(new_row, ignore_index=True)
        df_terrem = pd.concat([df_terrem, new_df], ignore_index = True)
#        print(f"Place: {properties}")
#        print(f"Place: {place}")
#        print(f"Magnitude: {magnitude}")
#        print(f"Coordinates: {coords}")

#        print("---")

nro_eventos = df_terrem.shape[0]

print('tenemos ',nro_eventos, 'eventos')

df_terrem.head()

tenemos  1229 eventos


,donde,lat,long,prof,t
0,"8km NW of The Geysers, CA",38.827168,-122.821167,1.69,1685402278400
1,"8km NW of The Geysers, CA",38.831833,-122.811333,2.07,1685398185030
2,"55 km S of Silver Peak, Nevada",37.253100,-117.613400,11.70,1685397934846
3,"7km WNW of The Geysers, CA",38.813333,-122.827000,1.92,1685395056990
4,"2km ESE of Home Gardens, CA",33.870167,-117.503333,9.84,1685394981560


### Creamos una matriz que contenga los valores de la fortaleza de los vínculos

Tomamos $\alpha = \beta_1 = 10^{-6}$ y $\epsilon_{1} = 1$

In [ ]:
mat_fv = np.zeros(shape=(nro_eventos,nro_eventos))

#nro_eventos = 25
alpha = 0.000001
beta = 0.000001
f_min = 1

for i in range(0,nro_eventos):

  lat_i = df_terrem.iloc[i]['lat']
  long_i = df_terrem.iloc[i]['long']
  prof_i = df_terrem.iloc[i]['prof']
  t_i = df_terrem.iloc[i]['t']
  for j in range(0,nro_eventos):

    if (j>i): # calculamos por arriba de la diagonal
      lat_j = df_terrem.iloc[j]['lat']
      long_j = df_terrem.iloc[j]['long']
      prof_j = df_terrem.iloc[j]['prof']
      t_j = df_terrem.iloc[j]['t']

      d = haversine_distance(lat_i, long_i, prof_i, lat_j, long_j, prof_j)
      f = 1/(alpha*d+beta*abs(t_i-t_j))

      if f > f_min:
        mat_fv[i,j] = round(f,4)
      #print(i,j,d,f)


In [ ]:
vinc = np.count_nonzero(mat_fv)
tot = nro_eventos ** 2
print(vinc,tot)

print(mat_fv)


1886 1510441
[[ 0.      0.      0.     ...  0.      0.      0.    ]
 [ 0.      0.      3.9893 ...  0.      0.      0.    ]
 [ 0.      0.      0.     ...  0.      0.      0.    ]
 ...
 [ 0.      0.      0.     ...  0.     12.3576  8.7145]
 [ 0.      0.      0.     ...  0.      0.     29.5586]
 [ 0.      0.      0.     ...  0.      0.      0.    ]]


In [ ]:


# Neo4j Aura credentials
bolt_url = "neo4j+s://6c289eb5.databases.neo4j.io"
username = "neo4j"
password = "4axXZ8PIGLXp-nAWF-IL2r1xNmfpnkGtf7dcHiJ9B50"

# Connect to the Neo4j Aura database
driver = GraphDatabase.driver(bolt_url, auth=(username, password))

# Create a session
with driver.session() as session:

    # Delete all nodes
    query = """
    MATCH (n)
    DETACH DELETE n
    """
    session.run(query)

    # Create nodes
    #nodes = [
    #    {"name": "Node 11", "property1": "1.001", "property2": 10},
    #    {"name": "Node 22", "property1": "2.002", "property2": 20},
    #    {"name": "Node 33", "property1": "3.003", "property2": 30}
    #]

    # Add nodes to the database
    #for node in nodes:
    #    query = """
    #    CREATE (n:Node {name: $name, property1: $property1, property2: $property2})
    #    """

    #    session.run(query, name=node["name"], property1=node["property1"], property2=node["property2"])

    #print("Nodes added successfully!")

 # Close the driver
#driver.close()

In [ ]:
nodos = []

for k in range(0,nro_eventos):

  lat_k = df_terrem.iloc[k]['lat']
  long_k = df_terrem.iloc[k]['long']
  prof_k = df_terrem.iloc[k]['prof']
  t_k = df_terrem.iloc[k]['t']
  nombre = "evento_" + str(k)

  nodos.append({"name":nombre,"lat":lat_k,"long":long_k,"prof":prof_k,"t":t_k})

  nodes = [
        {"name": "Node 11", "property1": "1.001", "property2": 10},
        {"name": "Node 22", "property1": "2.002", "property2": 20},
        {"name": "Node 33", "property1": "3.003", "property2": 30}
    ]



    # Add nodes to the database
# Create a session
with driver.session() as session:
  for node in nodos:
    query = """
    CREATE (n:Node {name: $nombre, lat: $lat, long: $long, prof:$prof, t:$t})
    """

    session.run(query, nombre=node["name"], lat=node["lat"], long=node["long"], prof=node["prof"], t=node["t"])

    #print("Nodes added successfully!")


In [ ]:
with driver.session() as session:

  for i in range(0,nro_eventos):

    for j in range(i+1,nro_eventos):

      if mat_fv[i,j] > 0:

        nombre_i = "evento_" + str(i)
        nombre_j = "evento_" + str(j)

        # Create a relationship between nodes
        create_relationship_query = """
        MATCH (n1:Node {name: $nombre_i})
        MATCH (n2:Node {name: $nombre_j})
        CREATE (n1)-[:REL_EVENTOS]->(n2)
        """

        session.run(create_relationship_query, nombre_i=nombre_i, nombre_j=nombre_j)

        #print("Relationship created successfully!")


In [ ]:
# Example usage

lat_montevideo = -34.883611
long_montevideo = -56.181944
lat_baires = -34.603333
long_baires =  -58.381667


lat1 = 37.7749
lon1 = -122.4194
depth1 = 0.0

lat2 = 34.0522
lon2 = -118.2437
depth2 = 0.0

lat1 = lat_montevideo
lon1 = long_montevideo
lat2 = lat_baires
lon2 = long_baires

distance = haversine_distance(lat1, lon1, depth1, lat2, lon2, depth2)
print(f"The distance between the two points is approximately {distance:.2f} kilometers.")


The distance between the two points is approximately 203.39 kilometers.
